In [ ]:
# transformers to load huggingface models
# datasets to load SQuAD dataset
# pandas for dataframe operations
!pip install transformers datasets pandas -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
import pandas as pd
import torch

In [ ]:
# Using the FLANT5 Base model for training
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


(…)b68e004ad934361fb35b9b2bd50b45ea90790fc8:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Google drive interface for saving the finetuned model
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Preprocessing function for SQuAD dataset
# Specifying the instruction format for the Input and Output
def preprocess_data(example):
    example["input_text"] = "question: " + example["question"] + " context: " + example["context"]
    example["target_text"] = example["answers"]["text"][0] if example["answers"]["text"] else "no answer"
    return example

In [ ]:
# Loading the SQuAD Dataset and preprocessing the train and validation portions
# Chunking the dataset due to Google Colab GPU hour limitations
dataset = load_dataset("squad")
train_data = dataset["train"].map(preprocess_data)
val_data = dataset["validation"].map(preprocess_data)
TRAIN_CHUNK = 25000
TEST_CHUNK = 5000
train_data = train_data.select(range(TRAIN_CHUNK))
val_data = val_data.select(range(TEST_CHUNK))

In [ ]:
# Specifying the max input and output lengths
# tokenizing the inputs and saving the targets as labels in the set.
max_input_length = 512
max_target_length = 128
def tokenize_function(example):
    inputs = tokenizer(example["input_text"], max_length=max_input_length, truncation=True, padding="max_length")
    targets = tokenizer(example["target_text"], max_length=max_target_length, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

In [ ]:
# Creating a tokenized version of the dataset and removing the column names
train_dataset = train_data.map(tokenize_function, batched=True, remove_columns=dataset["train"].column_names)
val_dataset = val_data.map(tokenize_function, batched=True, remove_columns=dataset["validation"].column_names)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
# if GPU is available, model loads on GPU. If not, it loads on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",  # Directory where model checkpoints and other outputs will be saved.
    evaluation_strategy="epoch",  # Perform evaluation at the end of each epoch.
    learning_rate=1e-4,  # The learning rate for the optimizer.
    per_device_train_batch_size=8,  # Number of samples per device (e.g., GPU) during training.
    per_device_eval_batch_size=8,  # Number of samples per device during evaluation.
    num_train_epochs=3,  # Number of complete passes through the training dataset.
    weight_decay=0.01,  # Strength of weight decay (L2 regularization) to prevent overfitting.
    # logging_dir="./logs",  # (Commented out) Directory for logging, useful for tools like TensorBoard and WandB.
    save_strategy="epoch",  # Save model checkpoints at the end of each epoch.
    report_to="none"  # Disable reporting to external tools like WandB or TensorBoard.
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Initialize the Seq2Seq trainer for training and evaluation
trainer = Seq2SeqTrainer(
    model=model,  # The sequence-to-sequence model to be trained
    args=training_args,  # Training arguments containing hyperparameters and configurations.
    train_dataset=train_dataset,  # The dataset used for training the model.
    eval_dataset=val_dataset,  # The dataset used for evaluating the model's performance during training.
)

# Start the training process
trainer.train()  # Trains the model using the specified arguments, training dataset, and evaluation dataset.


Epoch,Training Loss,Validation Loss
1,0.012500,0.014306
2,0.008600,0.015012


Epoch,Training Loss,Validation Loss
1,0.012500,0.014306
2,0.008600,0.015012
3,0.006300,0.015745


TrainOutput(global_step=9375, training_loss=0.012668695271809896, metrics={'train_runtime': 13506.3779, 'train_samples_per_second': 5.553, 'train_steps_per_second': 0.694, 'total_flos': 5.13568014336e+16, 'train_loss': 0.012668695271809896, 'epoch': 3.0})

In [ ]:
# Speficying the save path on Google Drive to save the model
save_path = "/content/drive/MyDrive/T5/fine_tuned_t5"
model.save_pretrained(save_path)